In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import os

In [ ]:
class ConcreteCrackDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_names = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert('RGB')
        label = 1 if 'crack' in self.img_names[idx] else 0  # Assuming filenames contain 'crack' for cracked images

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
class CrackDetectionCNN(nn.Module):
    def __init__(self):
        super(CrackDetectionCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 1)  # Change the final layer for binary classification

    def forward(self, x):
        return torch.sigmoid(self.model(x))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a manageable size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

dataset = ConcreteCrackDataset(img_dir='path_to_images', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CrackDetectionCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.float().to(device)

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

In [ ]:
torch.save(model.state_dict(), 'high-resolution-crack_detection_model-resnet-arch.pth')